# Structured Outputs

LLMs regurgitate out text and that is great for so many applications. But in order to build strong, robust systems and applications, we need to make sense of the chaos sometimes by receiving a pre-determined structured output everytime an LLM is called.

In [26]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display, Markdown


load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

# check if API keys are set
if not OPENAI_API_KEY:
    raise ValueError("Missing OpenAI API key")
if not GEMINI_API_KEY:
    raise ValueError("Missing Gemini API key")
if not ANTHROPIC_API_KEY:
    raise ValueError("Missing Anthropic API key")

In [27]:
from pydantic import BaseModel

class ProductReview(BaseModel):
    product: str
    review: str

class ReviewResponse(BaseModel):
    company: str
    moderate: str
    wittybot: str

class ReviewEvaluation(BaseModel):
    passed: bool
    feedback: str

In [28]:
user_message = "I want you to generate product review of a new generation of HP laptop series. "
user_message += "The review should be about user experience without too much technical details of the product. "

messages = [{"role": "user", "content": user_message}]

In [29]:
client = OpenAI()

openai_response = client.chat.completions.parse(
    model="gpt-4.1-nano",
    messages=messages,
    response_format=ProductReview
)

reviews = openai_response.choices[0].message.parsed

In [30]:
reviews.product

'HP NextGen Laptop Series'

In [31]:
reviews.review

"The new HP NextGen laptop series offers a sleek and modern design that feels premium to the touch. The user experience is seamless, with quick startup times and intuitive navigation. The keyboard is comfortable for extended typing sessions, and the display provides vibrant colors and sharp visuals. Navigating through applications is smooth, and the overall performance feels responsive and reliable. It's a great choice for both work and casual use, making daily tasks more enjoyable and efficient."

In [32]:
message = "You are to respond to the product review in 3 different styles (company, moderate and witty). \n\n"
message += f"Product: {reviews.product}\n"
message += f"Review: {reviews.review}\n\n"

messages = [{"role": "user", "content": message}]

In [33]:
response = client.chat.completions.parse(
    model="gpt-4.1-nano",
    messages=messages,
    response_format=ReviewResponse
)

response = response.choices[0].message.parsed

In [34]:
display(Markdown(response.company))

Thank you for sharing your positive experience with the HP NextGen Laptop Series. We're thrilled to hear that the design, performance, and display meet your expectations and enhance your daily tasks.

In [35]:
display(Markdown(response.moderate))

Thank you for your detailed review of the HP NextGen Laptop Series. We appreciate your feedback and are pleased to hear that you're satisfied with its design and performance. If you have any further comments or need assistance, feel free to reach out.

In [36]:
display(Markdown(response.wittybot))

Sounds like the HP NextGen Laptop Series has become your new best friend—sleek, reliable, and ready to make your daily grind a bit more glamorous. Thanks for the rave review!

In [ ]:
message = "You are to evaluate the proposed responses by company, witty bot, moderate for the following product review. "
message += "You will determine which response is appropriate for the product "
message += f"Product: {reviews.product}\n"
message += f"Review: {reviews.review}\n\n"
message += f"Company Response: {response.company}\n"
message += f"Moderate Response: {response.moderate}\n\n"
message += f"Witty Bot Response: {response.wittybot}\n\n"

messages = [{"role": "user", "content": message}]

In [44]:
evaluate = client.chat.completions.parse(
    model="gpt-4.1-nano",
    messages=messages,
    response_format=ReviewEvaluation
)

evaluate = evaluate.choices[0].message.parsed

In [46]:
print("Pass or not: ", evaluate.passed)

Pass or not:  True


In [45]:
#display(Markdown(evaluate.passed))
display(Markdown(f"Feedback: {evaluate.feedback}"))

Feedback: The witty bot response is appropriate and engages positively with the review, adding a touch of humor and enthusiasm that aligns well with the positive feedback.